# 1 start pods on Azure AKS

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-ksce2019`.

In [ ]:
export PJ_ROOT="${HOME}/example-ksce2019"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-ksce2019
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-ksce2019`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
if [ "$(uname)" == 'Darwin' ]; then
  alias randomstr8='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 8'
  alias randomstr16='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16'
  alias randomstr32='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32'
elif [ "$(expr substr $(uname -s) 1 5)" == 'Linux' ]; then
  alias randomstr8='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 8'
  alias randomstr16='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16'
  alias randomstr32='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32'
else
  echo "Your platform ($(uname -a)) is not supported."
  exit 1
fi

## add auth tokens for the web app of controller

### update secrets/auth-tokens.json

example)
```json
[
  {
    "host": "api\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "nrWtb8sS0MmwlkhHXv0DC6orPMpFFbni",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
        }
      ],
      "basic_auths": [],
      "no_auths": {}
    }
  },
  {
    "host": "kibana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "yW7FvSGD",
          "password": "6BoTFE5xfUlX3ssV",
          "allowed_paths": ["^.*$"]
        }
      ],
      "no_auths": {
        "allowed_paths": []
      }
    }
  },
  {
    "host": "grafana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": ["^.*$"]
      }
    }
  },
  {
    "host": "web\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "1JMF6D46",
          "password": "6u5M0bUhfjj7wMdM",
          "allowed_paths": [
            "^.*$"
          ]
        }
      ],
      "no_auths": {
        "allowed_paths": [
          "^.*/static/.*$"
        ]
      }
    }
  }
]
```

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.|=.+[{
  "host": "web\\..+$",
  "settings": {
    "bearer_tokens": [],
    "basic_auths": [
      {
        "username": "'$(randomstr8)'",
        "password": "'$(randomstr16)'",
        "allowed_paths": ["^.*$"]
      }
    ],
    "no_auths": {
      "allowed_paths": ["^.*/static/.*$"]
    }
  }
}]' | tee /tmp/auth-tokens.json
mv ${CORE_ROOT}/secrets/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json.back
mv /tmp/auth-tokens.json ${CORE_ROOT}/secrets/auth-tokens.json

## change the auth-tokens to kubernetes secrets

### watch auth log

Outside of this notebook

1. open a ternminal.
2. run a command displayed below.

In [ ]:
echo "kubectl logs -f -lapp=auth --all-containers=true"

### delete and re-register auth-tokens to kubernetes secrets

In [ ]:
kubectl delete secret auth-tokens

In [ ]:
kubectl create secret generic auth-tokens --from-file=${CORE_ROOT}/secrets/auth-tokens.json

### confirm the token will be reloaded

**wait a few minutes until the change of secret is detected by Kubernetes.**  
When the new secret is detected, the tokens of auth will be reloaded automatically.

log messages like below will be shown after tokens is reloaded.

```
...
--------
2019/09/25 06:55:27 hosts: [api\..+$ kibana\..+$ grafana\..+$ web\..+$]
--------
2019/09/25 06:55:27 bearerTokenAllowedPaths: map[api\..+$:map[a3K0mBgLIpXPmjzd7NtzEd1a28NRQD0e:[^/orion/.*$ ^/idas/.*$ ^/comet/.*$]]]
--------
2019/09/25 06:55:27 basicAuthPaths, map[kibana\..+$:map[^.*$:map[8njnClHL:7ivbWp1Qu5XbWPIt]] web\..+$:map[^.*$:map[EVu2A16R:If0aU5xZUiefG72D]]]
--------
2019/09/25 06:55:27 noAuthPaths, map[web\..+$:[^.*/static/.*$] api\..+$:[] kibana\..+$:[] grafana\..+$:[^.*$]]
--------
```

## start controller service on Azure AKS

In [ ]:
kubectl apply -f controller/ksce2019-controller-service.yaml

In [ ]:
kubectl get services -l app=controller

example)
```
NAME         TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
guide-demo   ClusterIP   10.0.236.157   <none>        3000/TCP   41s
```

## start controller pod on Azure AKS

set your API token of zaico.co,jp

In [ ]:
export ZAICO_TOKEN="xxxxxxxxxxxxxxxxxxxx"

In [ ]:
envsubst < controller/ksce2019-controller-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=controller

example)
```
NAME                          READY   STATUS    RESTARTS   AGE
controller-5ff54cdfdb-6jpzl   1/1     Running   0          72s
controller-5ff54cdfdb-frltr   1/1     Running   0          72s
controller-5ff54cdfdb-xp8v9   1/1     Running   0          72s
```

## register DNS A Record for business logic

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l app=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "web" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
curl -i https://web.${DOMAIN}/

example)
```
HTTP/1.1 404 Not Found
date: Wed, 25 Sep 2019 06:20:39 GMT
server: envoy
content-length: 0
```